<a href="https://colab.research.google.com/github/ixxan/ug-speech/blob/main/Fine_Tune_MMS_TTS_Uyghur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Meta MMS TTS for Uyghur with UQSpeech Dataset


Meta MMS TTS: https://huggingface.co/facebook/mms-tts

UQSpeech Dataset: https://github.com/gheyret/UQSpeechDataset

Finetune guide: https://github.com/ylacombe/finetune-hf-vits


In [ ]:
!pip install datasets umsc pydub jiwer evaluate transformers

In [ ]:
import torch
if torch.cuda.is_available():
  device = "cuda"
else:
  device = 'cpu'
device

'cuda'

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_TOKEN'))

hf_username = "ixxan"
repo_name = "mms-tts-uig-script_arabic-UQSpeech"

## Clone Finetune VITS Env

In [ ]:
# Clone the repository
!git clone https://github.com/ylacombe/finetune-hf-vits.git

# Change the working directory
%cd finetune-hf-vits

# Install requirements
!pip install -r requirements.txt

# Back to main directory
%cd ..

Cloning into 'finetune-hf-vits'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 173 (delta 49), reused 44 (delta 44), pack-reused 115 (from 1)
Receiving objects: 100% (173/173), 1.24 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/finetune-hf-vits
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take

## Build Cython Alignment

In [ ]:
%cd finetune-hf-vits/monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
%cd ..

/content/finetune-hf-vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/finetune-hf-vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure

## Convert MMS Checkpoint

In [ ]:
# %cd <path-to-finetune-hf-vits-repo>
# !python convert_original_discriminator_checkpoint.py \
#     --language_code <language-code> \
#     --pytorch_dump_folder_path <local-folder> \
#     --push_to_hub <repo-id-you-want>

In [ ]:
%cd finetune-hf-vits
!python convert_original_discriminator_checkpoint.py \
    --language_code uig-script_arabic \
    --pytorch_dump_folder_path "./mms-uig-script_arabic-checkpoint" \
    --push_to_hub "{hf_username}/{repo_name}"
%cd ..

/content/finetune-hf-vits
2024-12-25 01:42:18.347238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 01:42:18.366905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 01:42:18.372922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 01:42:18.387745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 01:42:19.636223:

## Prepare UQSpeech Data
Below is how I preprocessed UQSpeechData, the preprocessed data is already avaiable in HuggingFace: https://huggingface.co/datasets/ixxan/mms-tts-uig-script_arabic-UQSpeech, thus, you may skip this step. This is for reference only.

In [ ]:
# Download UQSpeech (Uyghur Quran Reading splited into small files)
!gdown "https://drive.google.com/uc?id=1sqcMf0Gl5FEiURQCQAV1SWW4R4f_VQt2"
!7z x UQSpeech.7z

Downloading...
From (original): https://drive.google.com/uc?id=1sqcMf0Gl5FEiURQCQAV1SWW4R4f_VQt2
From (redirected): https://drive.google.com/uc?id=1sqcMf0Gl5FEiURQCQAV1SWW4R4f_VQt2&confirm=t&uuid=3166ad6c-20a6-4949-a1ed-c75078d02675
To: /content/UQSpeech.7z
100% 2.57G/2.57G [00:43<00:00, 59.0MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2570754419 bytes (2452 MiB)

Extracting archive: UQSpeech.7z
--
Path = UQSpeech.7z
Type = 7z
Physical Size = 2570754419
Headers Size = 151426
Method = Delta LZMA2:24
Solid = +
Blocks = 4

  0%      0% 16 - UQSpeech/wavs/u0020005.wav                                      0% 36 - UQSpeech/wavs/u0020025.wav     

In [ ]:
import os
import pandas as pd

# Read metadata
UQSpeech_csv_path = 'UQSpeech/metadata.csv'
UQSpeech_df = pd.read_csv(UQSpeech_csv_path, delimiter='|', header=None)
UQSpeech_df = UQSpeech_df.iloc[:, [0, 1]] # Only keep file path and arabic script
UQSpeech_df.columns = ["path", "sentence"]
UQSpeech_df['path'] = 'UQSpeech/wavs/' + UQSpeech_df['path'] + '.wav'
UQSpeech_df

In [ ]:
# Convert to Dataset object
from datasets import Dataset, Audio

UQSpeech = Dataset.from_pandas(UQSpeech_df)
UQSpeech

In [ ]:
print(UQSpeech[1])
print(UQSpeech[4])

In [ ]:
# Remove punctuations from train set
import string

def remove_punctuation(batch):
    extra_punctuation = "–؛;،؟?«»‹›−—¬”“"  # Add your additional custom punctuation from the training set here
    all_punctuation = string.punctuation + extra_punctuation

    translator = str.maketrans('', '', all_punctuation)
    batch["sentence"] = batch["sentence"].translate(translator)
    return batch

UQSpeech = UQSpeech.map(remove_punctuation, num_proc=1)

print(UQSpeech[1])
print(UQSpeech[4])

In [ ]:
# Find the unique chars from UQSpeech
unique_chars = set()
for example in UQSpeech:
    unique_chars.update(example["sentence"])

print(sorted(list(unique_chars)))
print("Characters in train:", len(unique_chars))

# Fine the unique chars from MMS checkpoint
import json
with open('/content/finetune-hf-vits/mms-uig-script_arabic-checkpoint/vocab.json', 'r') as f:
    vocab_data = json.load(f)
print(sorted(list(vocab_data.keys())))
print("Characters in checkpoint vocab:", len(vocab_data.keys()))

chars_in_UQSpeech_not_in_vocab = unique_chars - vocab_data.keys()
chars_in_vocab_not_in_UQSpeech = vocab_data.keys() - unique_chars
print("Characters in train not in checkpoint vocab:", chars_in_UQSpeech_not_in_vocab)
print("Characters in checkpoint vocab not in train:", chars_in_vocab_not_in_UQSpeech)

In [ ]:
# Find the troublesome records
paths = set()
for c in chars_in_UQSpeech_not_in_vocab:
  for example in UQSpeech:
    if c in example["sentence"]:
      print(set(c))
      print(example["sentence"])
      paths.add(example['path'])
paths

In [ ]:
# Drop troublesome records
UQSpeech = UQSpeech.filter(lambda example: example["path"] not in paths)
UQSpeech

In [ ]:
# Cast the audio path to audio array
UQSpeech = UQSpeech.cast_column("path", Audio(sampling_rate=16000))

In [ ]:
# Optional: Push the dataset to hugging face
UQSpeech.push_to_hub(repo_name)

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Map:   0%|          | 0/1799 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ixxan/mms-tts-uig-script_arabic-UQSpeech/commit/584ea46910752b483645791bd94a1dbc2a5d869c', commit_message='Upload dataset', commit_description='', oid='584ea46910752b483645791bd94a1dbc2a5d869c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ixxan/mms-tts-uig-script_arabic-UQSpeech', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ixxan/mms-tts-uig-script_arabic-UQSpeech'), pr_revision=None, pr_num=None)

## Finetune

In [ ]:
# Define an finetune_uyghur.json as follow. Save the file to your working directory
# {
#     "project_name": "mms-tts-uig-script_arabic-finetune",
#     "push_to_hub": true,
#     "hub_model_id": "ixxan/mms-tts-uig-script_arabic-UQSpeech",
#     "overwrite_output_dir": true,
#     "output_dir": "./finetuned_uig_model",

#     "dataset_name": "ixxan/mms-tts-uig-script_arabic-UQSpeech",
#     "audio_column_name": "path",
#     "text_column_name":"sentence",
#     "train_split_name": "train",
#     "eval_split_name": "train",

#     "full_generation_sample_text": "مەككىدە نازىل بولغان يەتتە ئايەت",

#     "max_duration_in_seconds": 20,
#     "min_duration_in_seconds": 1.0,
#     "max_tokens_length": 500,

#     "model_name_or_path": "./finetune-hf-vits/mms-uig-script_arabic-checkpoint",

#     "preprocessing_num_workers": 4,

#     "do_train": true,
#     "num_train_epochs": 5,
#     "gradient_accumulation_steps": 1,
#     "gradient_checkpointing": false,
#     "per_device_train_batch_size": 16,
#     "learning_rate": 2e-5,
#     "adam_beta1": 0.8,
#     "adam_beta2": 0.99,
#     "warmup_ratio": 0.01,
#     "group_by_length": false,

#     "do_eval": true,
#     "eval_steps": 1000,
#     "per_device_eval_batch_size": 16,
#     "max_eval_samples": 25,
#     "do_step_schedule_per_epoch": true,

#     "weight_disc": 3,
#     "weight_fmaps": 1,
#     "weight_gen": 1,
#     "weight_kl": 1.5,
#     "weight_duration": 1,
#     "weight_mel": 35,

#     "fp16": true,
#     "seed": 456
# }

In [ ]:
!accelerate launch finetune-hf-vits/run_vits_finetuning.py ./finetune_uyghur.json

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-12-25 01:51:40.165791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 01:51:40.185450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 01:51:40.191407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alread

In [ ]:
from transformers import pipeline
import scipy
from IPython.display import Audio

model_id = "ixxan/mms-tts-uig-script_arabic-UQSpeech"
synthesiser = pipeline("text-to-speech", model_id, device=0) # add device=0 if you want to use a GPU

speech = synthesiser("ياق")

scipy.io.wavfile.write("finetuned_output.wav", rate=speech["sampling_rate"], data=speech["audio"][0])
Audio(speech["audio"][0], rate=16000)

Device set to use cuda:0
